<!-- # CNN autoencoder and Clustering from MTRX data

Use this notebook to load Scienta Omicron Matrix format SPM data and create standardised images for machine learning training and analysis. The code can generate both JPG image data, useful for manually checking the data, and windowed numpy data that can be loaded into ML models. 

The notebook then creates an autoencoder for training on a large dataset, followed by KMEANS clustering. 

**Author**: Steven R. Schofield  
**Created**: November, 2024 -->

# CASTEP - Si lattice parameter optimisations
## Steven R. Schofield (Universtiy College London) May 2025

### Load required modules

In [ ]:
import sys
from pathlib import Path

# Define candidate paths using Path objects
module_path_list = [
    Path('/Users/steven/academic-iCloud/Python/modules'),
    Path('/hpc/srs/Python/modules')
]

data_path_list = [
    Path('/Users/steven/academic-iCloud/Calculations/castep/'),
    Path('/hpc/srs/castep')
]

# Resolve actual paths
module_path = next((p for p in module_path_list if p.exists()), None)
data_path = next((p for p in data_path_list if p.exists()), None)

# Check and report missing paths
if module_path is None:
    print("Error: Could not locate a valid module path.")
if data_path is None:
    print("Error: Could not locate a valid data path.")

if module_path is None or data_path is None:
    sys.exit(1)

# Add module_path to sys.path if needed
if str(module_path) not in sys.path:
    sys.path.insert(0, str(module_path))

# Print resolved paths
print(f"module_path = {module_path}")
print(f"data_path = {data_path}")

In [ ]:
# # Ensure modules are reloaded 
%load_ext autoreload
%autoreload 2

# Import standard modules
import numpy as np
import pandas as pd
import os

# Import custom module
import SRSCALCUTILS.castep_tools as ct

from IPython.display import display, Image as StaticImage

### Job paths

In [ ]:
# Parameters for windows creation
# General
job_folder = 'work-2025/si_lat_apollo'
job_path = data_path / job_folder
verbose = False             # Set this True to print out more information

# Print resolved paths
print(f"job_path = {job_path}")

### Create CASTEP input files

In [ ]:
params = {'task' : 'geomopt',
          'xc_functional'           : 'PBE',
          'cut_off_energy'          : 700,
          'spin_polarised'          : 'false',
          'write_cell_structure'    : 'true',
          'charge'                  : 0,
          'nextra_bands'            : 6,
          'geom_energy_tol'         : 0.00005,          # default: 0.00005 eV 
          'geom_disp_tol'           : 0.002,            # default: 0.002 Ang.
          'geom_max_iter'           : 100,              # default: 30
          'geom_convergence_win'    : 2,                # default: 2
          'max_scf_cycles'          : 300               # default: 30
          }              

In [ ]:
# 

filename = "silicon"
title = "silicon lattice parameter optimisation"

na=1
nb=1
nc=1

lattice_cart = np.array([
            [3.8,     0.0,     0.0],
            [0.0,     3.8,     0.0],
            [0.0,     0.0,     5.4,]
        ])

positions_frac = np.array([
            ['Si', 0.0,  0.0,  0.0],
            ['Si', 0.5,  0.0,  0.25],
            ['Si', 0.5,  0.5,  0.5],
            ['Si', 0.0,  0.5,  0.75],
        ])

constraints = np.array([
            [1,  1,  2],
            [0,  0,  0]
        ])

kpoints_mp_grid = np.array([8, 8, 8])

fix_all_ions = True
symmetry_generate=False
symmetry_tol = 0.01

In [ ]:
# If it doesn’t exist, create it (including any missing parent folders)
if not os.path.isdir(job_path):
    os.makedirs(job_path, exist_ok=True)

# Now safely switch into it
os.chdir(job_path)

print("Now in:", os.getcwd())
print("Files:", os.listdir())

### Convergence with cut off energy

In [ ]:
!export OMP_NUM_THREADS=2

In [ ]:
var_list = [200,300,400,500,600,700,800,900,1000]
alist = []
energy_list = []

cores = 96

for var in var_list:
    params['cut_off_energy'] = var
    filename = f"silicon-{var}"
    print('\n' * 4)
    print('=' * 80)
    castep_file = str(job_path)+'/'+filename+'.castep'
    print('CASTEP file: {}'.format(castep_file))
    print('=' * 80)
    _ = ct.write_param_file(
        params,
        title = title,
        filename=filename,
        path=job_path,
        display_file=False
        )
    
    _ = ct.write_cell_file(
        title = title,
        path=job_path,
        filename=filename,
        na=na,
        nb=nb,
        nc=nc,
        lattice_cart=lattice_cart,
        positions_frac=positions_frac,
        constraints=constraints,
        fix_all_ions=True,
        symmetry_generate=symmetry_generate,
        symmetry_tol = symmetry_tol,
        kpoints_mp_grid=kpoints_mp_grid,
        display_file=False
    )
    
    
    if str(module_path) == '/hpc/srs/Python/modules':
        !bash -l -c "mpirun -np {cores} castep.mpi {filename}"
    else:
        !zsh -l -c "castepmpi {filename}"

    castep_path = job_path / f"{filename}.castep"

    warnings = ct.get_warnings(castep_path, verbose=True)

    calculation_parameters = ct.get_calculation_parameters(castep_path)
    calculation_parameters_df = pd.DataFrame(list(calculation_parameters.items()), columns=['Parameter', 'Value'])

    lattice_optimisation = ct.get_lattice_parameters(castep_path)
    a_values = [blk['a'] for blk in lattice_optimisation]

    unit_cell, a, b, c, alpha, beta, gamma = ct.get_final_lattice_parameters(castep_path)

    energy_optimisation = ct.get_LBFGS_energies(castep_path)
    energies = [val for _, val in energy_optimisation]

    ct.print_filename(castep_path)
    print(warnings)
    display(calculation_parameters_df)

    ct.plot_sequence(a_values, xlabel='Iteration', ylabel='a (Å)', title='Lattice parameter optimisation')
    ct.plot_sequence(energies, xlabel='Iteration', ylabel='Energy (eV)', title='Optimisation')

    print(f"Final unit cell")
    for vector in unit_cell:
        print(f"\t {vector}")
    print(f"Final a = {a:.4f} b = {b:.4f} c = {c:.4f} alpha = {alpha:.4f} beta = {beta:.4f} gamma = {gamma:.4f}\n")

    alist.append(a)
    energy_list.append(energies[-1])


ct.plot_sequence(energy_list, var_list, xlabel='cut_off_energy', ylabel='Energy (eV)', title='Optimisation')
ct.plot_sequence(alist, var_list, xlabel='cut_off_energy', ylabel='Lattice a (Ang.)', title='Optimisation')

import pandas as pd

df = pd.DataFrame({
    "Cut off energy (eV)": var_list,
    "Lattice constant (Å)": alist,
    "Energy": energy_list,
})

display( df)


In [ ]:
var_list = [200,300,400,500,600,700,800,900,1000]
alist = []
energy_list = []

cores = 64

for var in var_list:
    params['cut_off_energy'] = var
    filename = f"silicon-{var}"
    print('\n' * 4)
    print('=' * 80)
    castep_file = str(job_path)+'/'+filename+'.castep'
    print('CASTEP file: {}'.format(castep_file))
    print('=' * 80)
    _ = ct.write_param_file(
        params,
        title = title,
        filename=filename,
        path=job_path,
        display_file=False
        )
    
    _ = ct.write_cell_file(
        title = title,
        path=job_path,
        filename=filename,
        na=na,
        nb=nb,
        nc=nc,
        lattice_cart=lattice_cart,
        positions_frac=positions_frac,
        constraints=constraints,
        fix_all_ions=True,
        symmetry_generate=symmetry_generate,
        symmetry_tol = symmetry_tol,
        kpoints_mp_grid=kpoints_mp_grid,
        display_file=False
    )
    
    
    if str(module_path) == '/hpc/srs/Python/modules':
        !bash -l -c "mpirun -np {cores} castep.mpi {filename}"
    else:
        !zsh -l -c "castepmpi {filename}"

    castep_path = job_path / f"{filename}.castep"

    warnings = ct.get_warnings(castep_path, verbose=True)

    calculation_parameters = ct.get_calculation_parameters(castep_path)
    calculation_parameters_df = pd.DataFrame(list(calculation_parameters.items()), columns=['Parameter', 'Value'])

    lattice_optimisation = ct.get_lattice_parameters(castep_path)
    a_values = [blk['a'] for blk in lattice_optimisation]

    unit_cell, a, b, c, alpha, beta, gamma = ct.get_final_lattice_parameters(castep_path)

    energy_optimisation = ct.get_LBFGS_energies(castep_path)
    energies = [val for _, val in energy_optimisation]

    ct.print_filename(castep_path)
    print(warnings)
    display(calculation_parameters_df)

    ct.plot_sequence(a_values, xlabel='Iteration', ylabel='a (Å)', title='Lattice parameter optimisation')
    ct.plot_sequence(energies, xlabel='Iteration', ylabel='Energy (eV)', title='Optimisation')

    print(f"Final unit cell")
    for vector in unit_cell:
        print(f"\t {vector}")
    print(f"Final a = {a:.4f} b = {b:.4f} c = {c:.4f} alpha = {alpha:.4f} beta = {beta:.4f} gamma = {gamma:.4f}\n")

    alist.append(a)
    energy_list.append(energies[-1])


ct.plot_sequence(energy_list, var_list, xlabel='cut_off_energy', ylabel='Energy (eV)', title='Optimisation')
ct.plot_sequence(alist, var_list, xlabel='cut_off_energy', ylabel='Lattice a (Ang.)', title='Optimisation')

import pandas as pd

df = pd.DataFrame({
    "Cut off energy (eV)": var_list,
    "Lattice constant (Å)": alist,
    "Energy": energy_list,
})

display( df)


In [ ]:
var_list = [200,300,400,500,600,700,800,900,1000]
alist = []
energy_list = []

cores = 32

for var in var_list:
    params['cut_off_energy'] = var
    filename = f"silicon-{var}"
    print('\n' * 4)
    print('=' * 80)
    castep_file = str(job_path)+'/'+filename+'.castep'
    print('CASTEP file: {}'.format(castep_file))
    print('=' * 80)
    _ = ct.write_param_file(
        params,
        title = title,
        filename=filename,
        path=job_path,
        display_file=False
        )
    
    _ = ct.write_cell_file(
        title = title,
        path=job_path,
        filename=filename,
        na=na,
        nb=nb,
        nc=nc,
        lattice_cart=lattice_cart,
        positions_frac=positions_frac,
        constraints=constraints,
        fix_all_ions=True,
        symmetry_generate=symmetry_generate,
        symmetry_tol = symmetry_tol,
        kpoints_mp_grid=kpoints_mp_grid,
        display_file=False
    )
    
    
    if str(module_path) == '/hpc/srs/Python/modules':
        !bash -l -c "mpirun -np {cores} castep.mpi {filename}"
    else:
        !zsh -l -c "castepmpi {filename}"

    castep_path = job_path / f"{filename}.castep"

    warnings = ct.get_warnings(castep_path, verbose=True)

    calculation_parameters = ct.get_calculation_parameters(castep_path)
    calculation_parameters_df = pd.DataFrame(list(calculation_parameters.items()), columns=['Parameter', 'Value'])

    lattice_optimisation = ct.get_lattice_parameters(castep_path)
    a_values = [blk['a'] for blk in lattice_optimisation]

    unit_cell, a, b, c, alpha, beta, gamma = ct.get_final_lattice_parameters(castep_path)

    energy_optimisation = ct.get_LBFGS_energies(castep_path)
    energies = [val for _, val in energy_optimisation]

    ct.print_filename(castep_path)
    print(warnings)
    display(calculation_parameters_df)

    ct.plot_sequence(a_values, xlabel='Iteration', ylabel='a (Å)', title='Lattice parameter optimisation')
    ct.plot_sequence(energies, xlabel='Iteration', ylabel='Energy (eV)', title='Optimisation')

    print(f"Final unit cell")
    for vector in unit_cell:
        print(f"\t {vector}")
    print(f"Final a = {a:.4f} b = {b:.4f} c = {c:.4f} alpha = {alpha:.4f} beta = {beta:.4f} gamma = {gamma:.4f}\n")

    alist.append(a)
    energy_list.append(energies[-1])


ct.plot_sequence(energy_list, var_list, xlabel='cut_off_energy', ylabel='Energy (eV)', title='Optimisation')
ct.plot_sequence(alist, var_list, xlabel='cut_off_energy', ylabel='Lattice a (Ang.)', title='Optimisation')

import pandas as pd

df = pd.DataFrame({
    "Cut off energy (eV)": var_list,
    "Lattice constant (Å)": alist,
    "Energy": energy_list,
})

display( df)


In [ ]:
var_list = [200,300,400,500,600,700,800,900,1000]
alist = []
energy_list = []

cores = 12

for var in var_list:
    params['cut_off_energy'] = var
    filename = f"silicon-{var}"
    print('\n' * 4)
    print('=' * 80)
    castep_file = str(job_path)+'/'+filename+'.castep'
    print('CASTEP file: {}'.format(castep_file))
    print('=' * 80)
    _ = ct.write_param_file(
        params,
        title = title,
        filename=filename,
        path=job_path,
        display_file=False
        )
    
    _ = ct.write_cell_file(
        title = title,
        path=job_path,
        filename=filename,
        na=na,
        nb=nb,
        nc=nc,
        lattice_cart=lattice_cart,
        positions_frac=positions_frac,
        constraints=constraints,
        fix_all_ions=True,
        symmetry_generate=symmetry_generate,
        symmetry_tol = symmetry_tol,
        kpoints_mp_grid=kpoints_mp_grid,
        display_file=False
    )
    
    
    if str(module_path) == '/hpc/srs/Python/modules':
        !bash -l -c "mpirun -np {cores} castep.mpi {filename}"
    else:
        !zsh -l -c "castepmpi {filename}"

    castep_path = job_path / f"{filename}.castep"

    warnings = ct.get_warnings(castep_path, verbose=True)

    calculation_parameters = ct.get_calculation_parameters(castep_path)
    calculation_parameters_df = pd.DataFrame(list(calculation_parameters.items()), columns=['Parameter', 'Value'])

    lattice_optimisation = ct.get_lattice_parameters(castep_path)
    a_values = [blk['a'] for blk in lattice_optimisation]

    unit_cell, a, b, c, alpha, beta, gamma = ct.get_final_lattice_parameters(castep_path)

    energy_optimisation = ct.get_LBFGS_energies(castep_path)
    energies = [val for _, val in energy_optimisation]

    ct.print_filename(castep_path)
    print(warnings)
    display(calculation_parameters_df)

    ct.plot_sequence(a_values, xlabel='Iteration', ylabel='a (Å)', title='Lattice parameter optimisation')
    ct.plot_sequence(energies, xlabel='Iteration', ylabel='Energy (eV)', title='Optimisation')

    print(f"Final unit cell")
    for vector in unit_cell:
        print(f"\t {vector}")
    print(f"Final a = {a:.4f} b = {b:.4f} c = {c:.4f} alpha = {alpha:.4f} beta = {beta:.4f} gamma = {gamma:.4f}\n")

    alist.append(a)
    energy_list.append(energies[-1])


ct.plot_sequence(energy_list, var_list, xlabel='cut_off_energy', ylabel='Energy (eV)', title='Optimisation')
ct.plot_sequence(alist, var_list, xlabel='cut_off_energy', ylabel='Lattice a (Ang.)', title='Optimisation')

import pandas as pd

df = pd.DataFrame({
    "Cut off energy (eV)": var_list,
    "Lattice constant (Å)": alist,
    "Energy": energy_list,
})

display( df)


In [ ]:
var_list = [200,300,400,500,600,700,800,900,1000]
alist = []
energy_list = []

cores = 4

for var in var_list:
    params['cut_off_energy'] = var
    filename = f"silicon-{var}"
    print('\n' * 4)
    print('=' * 80)
    castep_file = str(job_path)+'/'+filename+'.castep'
    print('CASTEP file: {}'.format(castep_file))
    print('=' * 80)
    _ = ct.write_param_file(
        params,
        title = title,
        filename=filename,
        path=job_path,
        display_file=False
        )
    
    _ = ct.write_cell_file(
        title = title,
        path=job_path,
        filename=filename,
        na=na,
        nb=nb,
        nc=nc,
        lattice_cart=lattice_cart,
        positions_frac=positions_frac,
        constraints=constraints,
        fix_all_ions=True,
        symmetry_generate=symmetry_generate,
        symmetry_tol = symmetry_tol,
        kpoints_mp_grid=kpoints_mp_grid,
        display_file=False
    )
    
    
    if str(module_path) == '/hpc/srs/Python/modules':
        !bash -l -c "mpirun -np {cores} castep.mpi {filename}"
    else:
        !zsh -l -c "castepmpi {filename}"

    castep_path = job_path / f"{filename}.castep"

    warnings = ct.get_warnings(castep_path, verbose=True)

    calculation_parameters = ct.get_calculation_parameters(castep_path)
    calculation_parameters_df = pd.DataFrame(list(calculation_parameters.items()), columns=['Parameter', 'Value'])

    lattice_optimisation = ct.get_lattice_parameters(castep_path)
    a_values = [blk['a'] for blk in lattice_optimisation]

    unit_cell, a, b, c, alpha, beta, gamma = ct.get_final_lattice_parameters(castep_path)

    energy_optimisation = ct.get_LBFGS_energies(castep_path)
    energies = [val for _, val in energy_optimisation]

    ct.print_filename(castep_path)
    print(warnings)
    display(calculation_parameters_df)

    ct.plot_sequence(a_values, xlabel='Iteration', ylabel='a (Å)', title='Lattice parameter optimisation')
    ct.plot_sequence(energies, xlabel='Iteration', ylabel='Energy (eV)', title='Optimisation')

    print(f"Final unit cell")
    for vector in unit_cell:
        print(f"\t {vector}")
    print(f"Final a = {a:.4f} b = {b:.4f} c = {c:.4f} alpha = {alpha:.4f} beta = {beta:.4f} gamma = {gamma:.4f}\n")

    alist.append(a)
    energy_list.append(energies[-1])


ct.plot_sequence(energy_list, var_list, xlabel='cut_off_energy', ylabel='Energy (eV)', title='Optimisation')
ct.plot_sequence(alist, var_list, xlabel='cut_off_energy', ylabel='Lattice a (Ang.)', title='Optimisation')

import pandas as pd

df = pd.DataFrame({
    "Cut off energy (eV)": var_list,
    "Lattice constant (Å)": alist,
    "Energy": energy_list,
})

display( df)


In [ ]:
var_list = [200,300,400,500,600,700,800,900,1000]
alist = []
energy_list = []

cores = 1

for var in var_list:
    params['cut_off_energy'] = var
    filename = f"silicon-{var}"
    print('\n' * 4)
    print('=' * 80)
    castep_file = str(job_path)+'/'+filename+'.castep'
    print('CASTEP file: {}'.format(castep_file))
    print('=' * 80)
    _ = ct.write_param_file(
        params,
        title = title,
        filename=filename,
        path=job_path,
        display_file=False
        )
    
    _ = ct.write_cell_file(
        title = title,
        path=job_path,
        filename=filename,
        na=na,
        nb=nb,
        nc=nc,
        lattice_cart=lattice_cart,
        positions_frac=positions_frac,
        constraints=constraints,
        fix_all_ions=True,
        symmetry_generate=symmetry_generate,
        symmetry_tol = symmetry_tol,
        kpoints_mp_grid=kpoints_mp_grid,
        display_file=False
    )
    
    
    if str(module_path) == '/hpc/srs/Python/modules':
        !bash -l -c "mpirun -np {cores} castep.mpi {filename}"
    else:
        !zsh -l -c "castepmpi {filename}"

    castep_path = job_path / f"{filename}.castep"

    warnings = ct.get_warnings(castep_path, verbose=True)

    calculation_parameters = ct.get_calculation_parameters(castep_path)
    calculation_parameters_df = pd.DataFrame(list(calculation_parameters.items()), columns=['Parameter', 'Value'])

    lattice_optimisation = ct.get_lattice_parameters(castep_path)
    a_values = [blk['a'] for blk in lattice_optimisation]

    unit_cell, a, b, c, alpha, beta, gamma = ct.get_final_lattice_parameters(castep_path)

    energy_optimisation = ct.get_LBFGS_energies(castep_path)
    energies = [val for _, val in energy_optimisation]

    ct.print_filename(castep_path)
    print(warnings)
    display(calculation_parameters_df)

    ct.plot_sequence(a_values, xlabel='Iteration', ylabel='a (Å)', title='Lattice parameter optimisation')
    ct.plot_sequence(energies, xlabel='Iteration', ylabel='Energy (eV)', title='Optimisation')

    print(f"Final unit cell")
    for vector in unit_cell:
        print(f"\t {vector}")
    print(f"Final a = {a:.4f} b = {b:.4f} c = {c:.4f} alpha = {alpha:.4f} beta = {beta:.4f} gamma = {gamma:.4f}\n")

    alist.append(a)
    energy_list.append(energies[-1])


ct.plot_sequence(energy_list, var_list, xlabel='cut_off_energy', ylabel='Energy (eV)', title='Optimisation')
ct.plot_sequence(alist, var_list, xlabel='cut_off_energy', ylabel='Lattice a (Ang.)', title='Optimisation')

import pandas as pd

df = pd.DataFrame({
    "Cut off energy (eV)": var_list,
    "Lattice constant (Å)": alist,
    "Energy": energy_list,
})

display( df)
